In [1]:
import numpy as np
from scipy.stats import unitary_group


We will closely follow the math described by Robert Tucci in "An Introduction to Cartan's KAK Decomposition for QC Programmers"

First, we generate a random desired 4x4 unitary matrix transformation

In [2]:
X = unitary_group.rvs(4)

#Shape of X is 4x4
#Definition of unitary is that hermitian(U) @ U = I 
assert np.allclose(np.conj(X.T) @ X ,np.identity(4))
print("Matrix is unitary.")

Matrix is unitary.


Define some useful matrices

In [3]:
sig_x = np.array([[0,1],[1,0]])
sig_y = np.array([[0,-1j],[-1j,0]])
sig_z = np.array([[1,0],[0,-1]])
sig_1 = np.identity(2)
kron = lambda a,b=None: np.kron(a,b) if b else np.kron(a,a)
sigXX = kron(sig_x)
sigYY = kron(sig_y)
sigZZ = kron(sig_z)

#The Magic Basis
M = (1/np.sqrt(2)) * np.array(
    [[1,0,0,1j],
     [0,1j,1,0],
     [0,1j,-1,0],
     [1,0,0,-1j]]
)
E =  (1/np.sqrt(2)) * np.array(
    [[1,1j,0,0],
     [0,0,1j,1],
     [0,0,1j,-1],
     [1,-1j,0,0j]])



![alt text](image.png)

In [4]:
#phi = lambda A: np.conj(M.T) @ kron(A,np.conj(A)) @ M
#cap_phi = lambda A,B:np.conj(M.T) @ kron(A,np.conj(B)) @ M
X = unitary_group.rvs(4)

#Shape of X is 4x4
#Definition of unitary is that hermitian(U) @ U = I 
assert np.allclose(np.conj(X.T) @ X ,np.identity(4))
print("Matrix is unitary.")
X_prime = np.conj(M.T) @ X @ M

X_R =  np.real(X_prime)
X_I = np.imag(X_prime)
#find the simultaneous diagonalization of X_R and X_I 

Ua,S,Vta = np.linalg.svd(X_R)
Va = np.conj(Vta.T)

B_prime = Ua.T @ X_I @ Va

rankXR = np.linalg.matrix_rank(X_R)
G = B_prime[rankXR:,rankXR:]

evals, P = np.linalg.eig(G)

n = X_R.shape[0]
k = P.shape[0]

Pblock = np.block([
    [P,np.zeros((k, n - k))],
    [np.zeros((n - k, k)),np.identity(n - k)]
])

Q_L = Ua @ Pblock
Q_R = Va @ Pblock

eitheta = Q_L.T @ X_prime @ Q_R
assert np.allclose(eitheta, np.diag(np.diagonal(eitheta)), atol=1e-10)
print("Successfully diagonalized X_prime")


assert np.allclose(Q_L @ eitheta @ Q_R.T, X_prime)
print("Step 2 is complete")

K1= M @ Q_L @ np.conj(M.T)
K2 = (M @ Q_R @ np.conj(M.T)).conj().T
eik = M @ eitheta @ np.conj(M.T)

assert np.allclose(K1@eik@K2, X), "KAK decomposition is incorrect"
print("KAK decomposition is correct")

A = eik
sigx = np.matrix([[0,1],[1,0]])
sigz = np.matrix([[1,0],[0,-1]])
sigy = np.matrix([[0,-1j],[1j,0]])
gamma = lambda u: u @ np.kron(sigy,sigy) @ u.conj().T @ np.kron(sigy,sigy)  #IF WE EXPERIENCE ERRORS, IT MIGHT BE BECAUSE OF AMBIGUITY IN WHETHER TO USE CONJUGATE OR TRANSPOSE

gammaA = gamma(A)
eigvals  = np.linalg.eigvals(gammaA)
eigvals3 = eigvals[:3]

x,y,z = np.angle(eigvals3)

alpha = (x+y)/2
beta = (x+z)/2
delta = (y+z)/2

def Rz(theta):
    return np.array([
        [np.exp(-1j * theta / 2), 0],
        [0, np.exp(1j * theta / 2)]
    ])

# CNOT C₁₂ (control on qubit 0, target on qubit 1)
CN12 = np.array([
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0]
])


# Build A_canonical using your computed α, β, δ
A_canonical = (CN12 @ 
               np.kron(Rz(delta), Rz(beta)) @ 
               CN12 @ 
               np.kron(np.eye(2), Rz(alpha)))

assert(np.allclose(np.sort(np.angle(eigvals)),np.sort(np.angle(np.linalg.eigvals(gamma(A_canonical)))))), "Eigenvalues of gammaA and gammaA_canonical do not match, alpha beta delta are incorrect"
print("Valid alpha, beta, delta")


u_magic = E.conj().T @ A @ E
v_magic = E.conj().T @ A_canonical @ E

gamma_u = u_magic @ u_magic.conj().T
gamma_v = v_magic @ v_magic.conj().T

uEigvals, P = np.linalg.eigh(gamma_u)
vEigvals, Q = np.linalg.eigh(gamma_v)

R = np.linalg.inv(P.T@Q@v_magic)@u_magic

left = E@P.T @Q @ E.conj().T 
right = E@R @ E.conj().T

assert(np.allclose(left@A_canonical@right, A)), "Decomposition is incorrect"
print("Valid Decomposition of A into left, A_canonical, right ")

#sanity check 
assert(np.allclose(K1@left@A_canonical@right@K2, X))

AtensorB = K1@left 
CtensorD = right@K2
ItensorRy = np.kron(np.eye(2), Rz(alpha))
RztensorRy = np.kron(Rz(beta), Rz(delta))

assert(np.allclose(AtensorB@CN12@ItensorRy@CN12@RztensorRy@CtensorD, X))




Matrix is unitary.
Successfully diagonalized X_prime
Step 2 is complete
KAK decomposition is correct
Valid alpha, beta, delta
Valid Decomposition of A into left, A_canonical, right 


In [5]:
from qiskit.synthesis import TwoQubitBasisDecomposer
from qiskit.circuit.library import iSwapGate, CZGate
from qiskit.quantum_info import Operator
import numpy as np
from scipy.stats import unitary_group

iSwap = iSwapGate()
CZ = CZGate()

decomposer = TwoQubitBasisDecomposer(iSwap)
X = unitary_group.rvs(4)

circuit = decomposer(Operator(X))
print("Circuit:")
print(circuit)

relevant_matrices = []
tags = []

class operation:
    def __init__(self, gate, qubit_indices):

        gate_matrix = Operator(gate).data


        if len(qubit_indices) == 1:
            qubit_idx = qubit_indices[0]
      
            if qubit_idx == 0:
                relevant_matrices.append(gate_matrix)
                tags.append(0)
                self.U = np.kron(np.eye(2), gate_matrix)
            elif qubit_idx == 1:
                relevant_matrices.append(gate_matrix)
                tags.append(1)
                self.U = np.kron(gate_matrix, np.eye(2))
            else:
                assert False, f"Invalid qubit index: {qubit_idx}"
        else:
            self.U = gate_matrix
            relevant_matrices.append(gate_matrix)
            tags.append(2)

Useries = []
start = np.eye(4)
for CI in circuit.data:
    qubit_indices = [circuit.qubits.index(q) for q in CI.qubits]

    op = operation(CI.operation, qubit_indices)
    Useries.append(op.U)

    start = Useries[-1] @ start

# FIX 4: Apply global phase
start = np.exp(1j * circuit.global_phase) * start

assert np.allclose(start, X), "Decomposition is incorrect"



Circuit:
global phase: 4.4423
     ┌────────────────────────────┐┌────────┐ ┌────────────────────┐ ┌────────┐»
q_0: ┤ U(1.7248,-2.4446,0.076867) ├┤0       ├─┤ U(π/2,1.2682,-π/2) ├─┤0       ├»
     └┬─────────────────────────┬─┘│  Iswap │┌┴────────────────────┴┐│  Iswap │»
q_1: ─┤ U(1.2104,1.805,0.85941) ├──┤1       ├┤ U(0.036479,π/2,-π/2) ├┤1       ├»
      └─────────────────────────┘  └────────┘└──────────────────────┘└────────┘»
«     ┌──────────────────┐┌────────┐┌───────────────────────────┐
«q_0: ┤ U(1.6602,π/2,-π) ├┤0       ├┤ U(1.3116,-3.0663,-1.0206) ├
«     └┬───────────────┬─┘│  Iswap │└┬──────────────────────────┤
«q_1: ─┤ U(π/2,π/2,-π) ├──┤1       ├─┤ U(1.2292,-1.7467,2.3225) ├
«      └───────────────┘  └────────┘ └──────────────────────────┘


In [25]:
from typing import Any


def decompose_gate(U,mode): 
    iSwap = iSwapGate()
    CZ = CZGate()
    if mode=="CZ":
        relevant = CZ
    elif mode=="iSwap":
        relevant = iSwap
    decomposer = TwoQubitBasisDecomposer(relevant)
    circuit = decomposer(Operator(X))


    relevant_matrices = []
    tags = []

    
    def operation(gate, qubit_indices):

        gate_matrix = Operator(gate).data
        #relevant_matrices.append(gate_matrix)
        tag = None
        if gate_matrix.shape == (2,2):
            qubit_idx = qubit_indices[0]
            if qubit_idx == 0:
                tag = 0
                U = np.kron(np.eye(2), gate_matrix)
            elif qubit_idx == 1:
                tag = 1
                U = np.kron(gate_matrix, np.eye(2))
            else:
                assert False, f"Invalid qubit index: {qubit_idx}"
        else:
            U = gate_matrix
            tag = 2
        return U, gate_matrix, tag

    Useries = []
    start = np.eye(4)
    for CI in circuit.data:
        qubit_indices = [circuit.qubits.index(q) for q in CI.qubits]
        U, GM, tag = operation(CI.operation, qubit_indices)
        Useries.append(U)
        relevant_matrices.append(GM)
        tags.append(tag)

        start = Useries[-1] @ start

    # FIX 4: Apply global phase
    start = np.exp(1j * circuit.global_phase) * start

    assert np.allclose(start, X), "Decomposition is incorrect"

    return relevant_matrices,tags,circuit

X = unitary_group.rvs(4)

X = iSwapGate().to_matrix()
print(X)
X = np.array([[1+0.j, 0+0.j, 0.+0.j, 0.+0.j],
              [0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j],
              [0.+0.j, 0.+1.j, 0.+0.j, 0.+0.j],
              [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])
from dataclasses import dataclass

@dataclass
class physical_instruction:
    code: str
    title: str
    tag: int
    underlying_gate: Any
    instruction_string: str
    details: str
    


from qiskit.synthesis import OneQubitEulerDecomposer

def round3(x):
    return np.round(x,3)
def InstructionSet(RM,tags, mode, rparams):

    decomposer_zyz = OneQubitEulerDecomposer(basis='ZYZ')
    InstructionSet = []
    
    for i in range(len(RM)):

        if tags[i] == 2:
            if mode=="iSwap":
                code = "ISWAP"
                title = "iSwap Gate"
                underlying_gate = iSwap
                instruction_string = f"Tune Qubit 1 to the frequency of Qubit 0 for <time>"
                details = f"This is a physical operation that realizes the iSwap gate. Apply a DC flux pulse over the transmon's SQUID loop to modify the flux through the loop and change the qubits drive frequency. By placing the two qubits in resonance, the natural entangling interaction between the two qubits is activated and realizes the iSwap gate over time. "
                #param format: [time, time_label]
               
                InstructionSet.append(physical_instruction(code,title,tags[i], underlying_gate, instruction_string, details))
               
            elif mode=="CZ":
                assert False, "CZ gate is not implemented yet."
            
        else:
           
            ZYZ = decomposer_zyz(RM[i])
            #sometimes ZYZ is in format YZ or ZY or just Z or just Y . we must check for this
            one_on = True
            two_on = True
            three_on = True
            if(len(ZYZ.data) == 0):
                one_on = False
                two_on = False
                three_on = False
            if(len(ZYZ.data) == 1):
                if(ZYZ.data[0].operation.name == "rz"):
                    #gate is just a virtual Z rotation
                    one_on = True
                    two_on = False
                    three_on = False
                else:
                    one_on = False
                    two_on = True
                    three_on = False
            if(len(ZYZ.data) == 2):
                if(ZYZ.data[0].operation.name == "rz"):
                    one_on = True
                    two_on = True
                    three_on = False
                else:
                    one_on = False
                    two_on = True
                    three_on = True

            if tags[i] == 0:
                count = 0 
                #gate 1 (virtual Z rotation)
                if(one_on):
                    instruction,angle1 = genZInstruction(ZYZ.data[count],rparams,0)
                    rparams.q0current_relative_phase += angle1
                    InstructionSet.append(instruction)
                    count += 1
                #gate 2 (Y gate)
                if(two_on):
                    instruction2 = genYInstruction(ZYZ.data[count],rparams,0)
                    InstructionSet.append(instruction2)
                    count += 1
                #gate 3 (virtual Z rotation)
                if(three_on):
                    instruction3,angle3= genZInstruction(ZYZ.data[count],rparams,0)
                    rparams.q0current_relative_phase += angle3
                    InstructionSet.append(instruction3)
                    count += 1

            elif tags[i] == 1:
                #gate 1 (virtual Z rotation)
                count = 0 
                if(one_on):
                    instruction,angle3 = genZInstruction(ZYZ.data[count],rparams,1)
                    rparams.q1current_relative_phase += angle3
                    InstructionSet.append(instruction)
                    count += 1
                #gate 2 (Y gate)
                if(two_on):
                    instruction2 = genYInstruction(ZYZ.data[count],rparams,1)
                    InstructionSet.append(instruction2)
                    count += 1
                #gate 3 (virtual Z rotation)
                if(three_on):
                    instruction3,angle3 = genZInstruction(ZYZ.data[count],rparams,1)
                    rparams.q1current_relative_phase += angle3
                    InstructionSet.append(instruction3)
                    count += 1
     
    return InstructionSet,rparams

def genZInstruction(circuitinstruction,rparams,tag):
    g = circuitinstruction.operation
    angle = g.params[0]
    gatem = Operator(g).data
    code = "RZ"
    title = f"Z-axis rotation of ~{round3(angle)} radians on Qubit {tag}"
    if tag == 0:
        underlying_gate = np.kron(np.eye(2), gatem)
    elif tag == 1:
        underlying_gate = np.kron(gatem, np.eye(2))
    else:
        assert False, f"Invalid qubit index: {tag}"
    instruction_string = f"Virtual (bookkeeping) Z-axis phase rotation"
    details = f"Phase rotations do not need to be applied physically. By shifting the phase our computational basis, we can effectively apply a Z rotation on Qubit 0 in 0 time. All future applied radiation will be prepared with a phase shift that matches how much we virtually rotated around the Z-axis."
    return physical_instruction(code,title,tag, underlying_gate, instruction_string, details),angle
def genYInstruction(circuitinstruction,rparams,tag):
    g = circuitinstruction.operation
    
    gatem = Operator(g).data
    code = "RY"
    angle = g.params[0]
    title = f"Y-axis rotation of ~{round3(angle)} radians on Qubit {tag}"
 
    if tag == 0:
        qdrive_freq = rparams.q0drive_freq
        underlying_gate = np.kron(np.eye(2), gatem)
        usephase = rparams.q0current_relative_phase
    elif tag == 1:
        qdrive_freq = rparams.q1drive_freq
        underlying_gate = np.kron(gatem, np.eye(2))
        usephase = rparams.q1current_relative_phase
    else:
        assert False, f"Invalid qubit index: {tag}"
    time = angle/rparams.rabi_frequency
    instruction_string = f"Apply electromagnetic radiation to qubit {tag} through it's Drive capacitor "
    details = f"""Applying a rotation around the Y axis is via phase = PI/2, since this phase produces a pure Pauli Y rotation in the drive hamiltonian. The amplitude of the drive is k*rabi_frequency, where k is a constant must be calibrated on a real system. We must also factor in our current relative phase (so that the virtual Z rotations are considered). The speed of this rotation is dictated by the Rabi Frequency (time =theta/rabi)

Drive Frequency: {round3(qdrive_freq*(1e-9))} GHz 
Phase: PI/2 + Current Relative Phase: {round3(np.pi/2 + usephase)} radians 
Amplitude: k* {round3(rparams.rabi_frequency*(1e-6))} MHz 
Time: {round3(time*1e9)} ns"""
    return physical_instruction(code,title, tag, underlying_gate, instruction_string, details)
@dataclass
class relevant_parameters:
    rabi_frequency: float 
    q0drive_freq: float
    q1drive_freq: float
    q0current_relative_phase: float
    q1current_relative_phase: float

rparams = relevant_parameters(rabi_frequency=20e6, q0drive_freq=5.3e9, q1drive_freq=5e9, q0current_relative_phase=0.0,q1current_relative_phase=0.0)
mode = "iSwap" # "CZ"
RM, tags,qcircuit = decompose_gate(X,mode)
instructionset, final_rparams =  InstructionSet(RM,tags,mode,rparams)


len(instructionset)
print([(instruction.code,instruction.tag) for instruction in instructionset])
!pip install pylatexenc
print(qcircuit.draw(reverse_bits=True,output="mpl",filename="assets/circuit_image/circuit.png"))

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]
[('ISWAP', 2)]
Figure(287.294x200.667)


In [7]:
decomposer_zyz = OneQubitEulerDecomposer(basis='ZYZ')
decomposer_zyz(np.eye(2)).data

[]

In [8]:
#TESTING THE MEANING OF PARAMETERS OF A U GATE IN QISKIT
from qiskit import QuantumCircuit
from qiskit.circuit.library import iSwapGate, CZGate, RZGate, RYGate, RXGate

from qiskit.transpiler import PassManager
import math
qc = QuantumCircuit(1)
qc.u(math.pi/2,math.pi/4,math.pi/5,qubit=0)
Operator(qc.data[0].operation).data
qc.data[0].operation.params
unitary = Operator(qc).data
decomposer_zyz = OneQubitEulerDecomposer(basis='ZYZ')
circuit_zyz = decomposer_zyz(unitary)
qc2 = QuantumCircuit(2)
qc2.rz(math.pi/5,0)
qc2.ry(math.pi/2,0)
qc2.rz(math.pi/4,0)
print(circuit_zyz.draw())
print(qc2.draw())
qc2.global_phase = qc2.global_phase + circuit_zyz.global_phase
print(Operator(qc2).data)

print(Operator(circuit_zyz).data)



global phase: 0.70686
   ┌─────────┐┌─────────┐┌─────────┐
q: ┤ Rz(π/5) ├┤ Ry(π/2) ├┤ Rz(π/4) ├
   └─────────┘└─────────┘└─────────┘
     ┌─────────┐┌─────────┐┌─────────┐
q_0: ┤ Rz(π/5) ├┤ Ry(π/2) ├┤ Rz(π/4) ├
     └─────────┘└─────────┘└─────────┘
q_1: ─────────────────────────────────
                                      
[[ 0.70710678-1.57344811e-17j -0.5720614 -4.15626938e-01j
   0.        +0.00000000e+00j  0.        +0.00000000e+00j]
 [ 0.5       +5.00000000e-01j  0.11061587+6.98401123e-01j
   0.        +0.00000000e+00j  0.        +0.00000000e+00j]
 [ 0.        +0.00000000e+00j  0.        +0.00000000e+00j
   0.70710678-1.57344811e-17j -0.5720614 -4.15626938e-01j]
 [ 0.        +0.00000000e+00j  0.        +0.00000000e+00j
   0.5       +5.00000000e-01j  0.11061587+6.98401123e-01j]]
[[ 0.70710678-2.70972568e-18j -0.5720614 -4.15626938e-01j]
 [ 0.5       +5.00000000e-01j  0.11061587+6.98401123e-01j]]


In [9]:
print(qc.data[0])
print(qc2.data[1].operation.params)
M = Operator(qc2.data[1].operation).data

assert(np.allclose(np.real(M), M))
M = np.real(M)
theta = 2 * np.arctan2(M[1,0], M[0,0])
print(theta)

CircuitInstruction(operation=Instruction(name='u', num_qubits=1, num_clbits=0, params=[1.5707963267948966, 0.7853981633974483, 0.6283185307179586]), qubits=(<Qubit register=(1, "q"), index=0>,), clbits=())
[1.5707963267948966]
1.5707963267948963


In [10]:
import qiskit.quantum_info as qi

qc = QuantumCircuit(2)
#cnot 1st qubit is control, 2nd qubit is target
qc.cx(0,1)
print(qi.Operator(qc))
#cnot 1st qubit is target, 2nd qubit is control
qc = QuantumCircuit(2)
qc.cx(1,0)
print(qi.Operator(qc))


qc = QuantumCircuit(2)
qc.h(0)
print(qi.Operator(qc).data)

hadamardMat = np.array([[1,1],[1,-1]])/np.sqrt(2)
print(np.kron(np.eye(2),hadamardMat))

#in qiskit, the bottom qubit is qubit 0. 




Operator([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
          [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
Operator([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
          [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
[[ 0.70710678+0.j  0.70710678+0.j  0.        +0.j  0.        +0.j]
 [ 0.70710678+0.j -0.70710678+0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.70710678+0.j  0.70710678+0.j]
 [ 0.        +0.j  0.        +0.j  0.70710678+0.j -0.70710678+0.j]]
[[ 0.70710678  0.70710678  0.          0.        ]
 [ 0.70710678 -0.70710678  0.         -0.        ]
 [ 0.          0.          0.70710678  0.70710678]
 [ 0.         -0.          0.70710678 -0.70710678]]


In [ ]:
qc = QuantumCircuit(1)
qc.h(0)
hadamardMat =qi.Operator(qc).data
result = hadamardMat @ np.array([[1],[0]])

qc = QuantumCircuit(2)
tensor = np.kron(np.eye(2),hadamardMat)

result2 = tensor @ np.array([[1],[0],[0],[]])

print(result)
print(result2)




[[0.70710678+0.j]
 [0.70710678+0.j]]
[[ 0.        +0.j]
 [ 0.        +0.j]
 [ 0.70710678+0.j]
 [-0.70710678+0.j]]
